# Sélection des bandes discriminantes pour les images hyperspectrales
# Méthode : IOU VS ES IOU

**Dans ce notebook, nous développons une méthode de classification multiclasse en utilisant des réseaux de neurones MLP avec une approche de sélection de bandes basée sur l'analyse "worst-case". Nous chargeons les résultats de séparabilité calculés précédemment et implémentons deux stratégies de sélection : la première sélectionne les meilleures bandes selon leur score worst-case global (top 5, 10, 15, 20), tandis que la seconde utilise une approche "equal spacing" qui divise le spectre en segments égaux et sélectionne la meilleure bande de chaque segment. Nous entraînons des modèles MLP avec architecture 512-64-64-32 incluant BatchNormalization et Dropout pour la classification de toutes les classes simultanément (incluant le background), puis comparons les performances en termes d'accuracy, temps d'entraînement et F1-score par classe pour déterminer la stratégie de sélection de bandes la plus efficace pour la classification multiclasse.**

### Dataset : Indian Pines 

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import loadmat
from tqdm import tqdm
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, BatchNormalization, Input, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import os

In [15]:
# Définir le chemin du dataset
dataset_path = "/kaggle/input/dataset-indian"  # Chemin vers le dossier du dataset

In [16]:
# Affichage des variables 
print("Variables dans Indian_pines.mat:")
chemin_image = os.path.join(dataset_path, "Indian_pines.mat")
try:
    donnees_mat = loadmat(chemin_image)
    image_keys = [key for key in donnees_mat.keys() if not key.startswith('__')]
    print(image_keys)
    print(f"Forme des données: {donnees_mat[image_keys[0]].shape if len(image_keys) > 0 else 'N/A'}")
except Exception as e:
    print(f"Erreur lors du chargement: {e}")

# Afficher les variables du fichier d'image corrigée
print("\nVariables dans Indian_pines_corrected.mat:")
chemin_image_corr = os.path.join(dataset_path, "Indian_pines_corrected.mat")
try:
    donnees_mat_corr = loadmat(chemin_image_corr)
    image_corr_keys = [key for key in donnees_mat_corr.keys() if not key.startswith('__')]
    print(image_corr_keys)
    print(f"Forme des données: {donnees_mat_corr[image_corr_keys[0]].shape if len(image_corr_keys) > 0 else 'N/A'}")
except Exception as e:
    print(f"Erreur lors du chargement: {e}")

# Afficher les variables du fichier de vérité terrain
print("\nVariables dans Indian_pines_gt.mat:")
chemin_gt = os.path.join(dataset_path, "Indian_pines_gt.mat")
try:
    gt_mat = loadmat(chemin_gt)
    gt_keys = [key for key in gt_mat.keys() if not key.startswith('__')]
    print(gt_keys)
    print(f"Forme des données: {gt_mat[gt_keys[0]].shape if len(gt_keys) > 0 else 'N/A'}")
except Exception as e:
    print(f"Erreur lors du chargement: {e}")

Variables dans Indian_pines.mat:
['indian_pines']
Forme des données: (145, 145, 220)

Variables dans Indian_pines_corrected.mat:
['indian_pines_corrected']
Forme des données: (145, 145, 200)

Variables dans Indian_pines_gt.mat:
['indian_pines_gt']
Forme des données: (145, 145)


###   ÉTAPE 1: Charger les données hyperspectrales et les vérités terrain

In [17]:
def charger_donnees(dataset_path):
    """
    Charge les images hyperspectrales et les vérités terrain.
    
    Args:
        dataset_path: Chemin vers le dossier contenant les fichiers .mat
    
    Returns:
        donnees_hyperspectrales: Données spectrales (n_rows, n_cols, n_bands)
        verite_terrain: Vérités terrain (n_rows, n_cols)
    """
    print("Chargement des données hyperspectrales...")
    
    # Chargement des données corrigées
    chemin_image = os.path.join(dataset_path, "Indian_pines_corrected.mat")
    donnees_mat = loadmat(chemin_image)
    
    # Utiliser le nom de variable exact pour l'image hyperspectrale
    donnees_hyperspectrales = donnees_mat['indian_pines_corrected']  # Variable: 'indian_pines_corrected'
    print(f"Dimensions de l'image hyperspectrale: {donnees_hyperspectrales.shape}")
    
    # Charger la vérité terrain
    chemin_gt = os.path.join(dataset_path, "Indian_pines_gt.mat")
    gt_mat = loadmat(chemin_gt)
    
    # Utiliser le nom de variable exact pour la vérité terrain
    verite_terrain = gt_mat['indian_pines_gt']  # Variable: 'indian_pines_gt'
    print(f"Dimensions de la vérité terrain: {verite_terrain.shape}")
    
    # Afficher des informations sur les données chargées
    print(f"Nombre de classes uniques dans la vérité terrain: {len(np.unique(verite_terrain))}")
    print(f"Classes uniques: {np.unique(verite_terrain)}")
    
    return donnees_hyperspectrales, verite_terrain

### ÉTAPE 2: Préparer les données pour l'analyse

In [18]:
def preparer_donnees(donnees_hyperspectrales, verite_terrain):
    """
    Prépare les données pour l'analyse de séparabilité.
    
    Args:
        donnees_hyperspectrales: Données spectrales (n_rows, n_cols, n_bands)
        verite_terrain: Vérités terrain (n_rows, n_cols)
    
    Returns:
        pixels: Données spectrales (n_pixels, n_bands)
        classes: Étiquettes de classe pour chaque pixel (n_pixels)
        classes_uniques: Liste des classes uniques
        class_names: Noms des classes
    """
    # Définir les noms des classes pour Indian Pines
    # Les 16 classes + background (classe 0) d'après la documentation
    class_names = [
        "Background",             # 0
        "Alfalfa",                # 1
        "Corn-notill",            # 2
        "Corn-mintill",           # 3
        "Corn",                   # 4
        "Grass-pasture",          # 5
        "Grass-trees",            # 6
        "Grass-pasture-mowed",    # 7
        "Hay-windrowed",          # 8
        "Oats",                   # 9
        "Soybean-notill",         # 10
        "Soybean-mintill",        # 11
        "Soybean-clean",          # 12
        "Wheat",                  # 13
        "Woods",                  # 14
        "Buildings-Grass-Trees-Drives", # 15
        "Stone-Steel-Towers"      # 16
    ]
    
    # Obtenir les dimensions des données
    height, width, n_bands = donnees_hyperspectrales.shape
    
    # Réorganiser les données pour l'analyse
    pixels = donnees_hyperspectrales.reshape(height * width, n_bands)
    classes = verite_terrain.reshape(height * width)
    
    # Extraire les classes uniques (y compris le background - classe 0)
    classes_uniques = np.unique(classes)
    
    print(f"Données préparées: {pixels.shape[0]} pixels avec {pixels.shape[1]} bandes")
    print(f"Nombre de classes (avec background): {len(classes_uniques)}")
    
    # Compter le nombre de pixels par classe
    for classe in classes_uniques:
        n_pixels = np.sum(classes == classe)
        nom_classe = class_names[classe] if classe < len(class_names) else f"Classe {classe}"
        print(f"Classe {classe} ({nom_classe}): {n_pixels} pixels")
    
    return pixels, classes, classes_uniques, class_names

### ÉTAPE 3: Fonction de calcul du chevauchement

In [19]:
def calculer_chevauchement(classe_A_min, classe_A_max, classe_B_min, classe_B_max):
    """
    Calcule le chevauchement entre deux plages de valeurs.
    
    Args:
        classe_A_min, classe_A_max: Valeurs min et max pour la classe A
        classe_B_min, classe_B_max: Valeurs min et max pour la classe B
    
    Returns:
        Chevauchement normalisé (0 signifie aucun chevauchement, valeur positive indique un chevauchement)
    """
    # Calcul des bornes de chevauchement
    a = max(classe_A_min, classe_B_min)  # La plus grande des valeurs minimales
    b = min(classe_A_max, classe_B_max)  # La plus petite des valeurs maximales
    
    # Calcul du chevauchement brut
    c = b - a
# Calcul de l'étendue totale
    etendue_totale = max(classe_A_max, classe_B_max) - min(classe_A_min, classe_B_min)
    
    # Normalisation du chevauchement
    if etendue_totale > 0:
        c_normalise = c / etendue_totale
    else:
        c_normalise = 0
    
    # Retourne max(0, c_normalise)
    return max(0, c_normalise)

### ÉTAPE 4: Fonction principale pour calculer la séparabilité entre paires de classes

In [20]:
def calculer_separabilite_paires_classes(pixels, classes, classes_uniques, class_names):
    """
    Calcule la séparabilité entre chaque paire de classes pour chaque bande spectrale.
    
    Args:
        pixels: Données spectrales (n_pixels x n_bandes)
        classes: Étiquettes de classe pour chaque pixel
        classes_uniques: Liste des classes uniques à considérer
        class_names: Noms des classes
    
    Returns:
        DataFrame contenant les résultats de séparabilité par paires
    """
    # Créer une liste pour stocker les résultats
    resultats_list = []
    
    # Nombre total d'itérations pour la barre de progression
    total_iterations = len(classes_uniques) * (len(classes_uniques) - 1) // 2 * pixels.shape[1]
    
    # Utiliser tqdm pour afficher une barre de progression
    with tqdm(total=total_iterations, desc="Calcul de séparabilité par paires") as pbar:
        # Pour chaque paire de classes
        for i, classe_A in enumerate(classes_uniques):
            for classe_B in classes_uniques[i+1:]:  # Ne considérer que les paires uniques
                # Obtenir les noms des classes
                nom_classe_A = class_names[classe_A] if classe_A < len(class_names) else f"Classe {classe_A}"
                nom_classe_B = class_names[classe_B] if classe_B < len(class_names) else f"Classe {classe_B}"
                
                # Créer les masques pour les deux classes
                mask_A = classes == classe_A
                mask_B = classes == classe_B
                
                # Pour chaque bande spectrale
                for bande in range(pixels.shape[1]):
                    # Extraire les valeurs de la bande pour les deux classes
                    valeurs_A = pixels[mask_A, bande]
                    valeurs_B = pixels[mask_B, bande]
                    
                    # Vérifier que les deux classes ont des pixels
                    if len(valeurs_A) > 0 and len(valeurs_B) > 0:
                        # Calculer les min et max pour chaque classe
                        classe_A_min = np.min(valeurs_A)
                        classe_A_max = np.max(valeurs_A)
                        classe_B_min = np.min(valeurs_B)
                        classe_B_max = np.max(valeurs_B)
                        
                        # Calculer le chevauchement normalisé
                        chevauchement = calculer_chevauchement(classe_A_min, classe_A_max, classe_B_min, classe_B_max)
                        
                        # Calculer la séparabilité (1 - chevauchement)
                        separabilite = 1 - chevauchement
                        
                        # Stocker les résultats dans la liste
                        resultats_list.append({
                            'ClasseA': int(classe_A),
                            'ClasseB': int(classe_B),
                            'NomClasseA': nom_classe_A,
                            'NomClasseB': nom_classe_B,
                            'Bande': int(bande),
                            'Separabilite': float(separabilite),
                            'Chevauchement': float(chevauchement)
                        })
                    
                    # Mettre à jour la barre de progression
                    pbar.update(1)
    
    # Créer un DataFrame à partir de la liste de résultats
    resultats_paires = pd.DataFrame(resultats_list)
    
    return resultats_paires

### ÉTAPE 5: Analyser et trier les bandes selon leur worst-case 

In [21]:
def analyser_worst_case(resultats_paires, top_n=20):
    """
    Analyse les bandes par leur worst-case de séparabilité et affiche les top N.
    
    Args:
        resultats_paires: DataFrame contenant les résultats de séparabilité
        top_n: Nombre de meilleures bandes à afficher
    
    Returns:
        DataFrame contenant les bandes triées par leur worst-case
    """
    print("Identification du worst-case pour chaque bande...")
    
    # Pour chaque bande, trouver le pire cas de séparabilité (minimum)
    worst_case_par_bande = (resultats_paires
                           .groupby('Bande')
                           .agg({
                               'Separabilite': 'min',  # Prendre la séparabilité minimum
                               'Chevauchement': 'max'   # Le chevauchement maximum correspondant
                           })
                           .reset_index())

    # Trier les bandes par leur worst-case de séparabilité (ordre décroissant)
    worst_case_par_bande = worst_case_par_bande.sort_values('Separabilite', ascending=False)

    # Afficher les top_n meilleures bandes selon leur worst-case
    print(f"\nTop {top_n} des bandes selon leur pire cas de séparabilité:")
    print(worst_case_par_bande.head(top_n))
    
    # Sauvegarder le tableau complet
    worst_case_par_bande.to_csv('worst_case_par_bande.csv', index=False)
    print(f"Tableau des worst-case par bande sauvegardé dans 'worst_case_par_bande.csv'")
    
    return worst_case_par_bande


### ÉTAPE 6: Fonction principale pour exécuter l'analyse complète.

In [22]:
def main():
    """
    Fonction principale pour exécuter l'analyse complète.
    """
    # Créer un dossier de sortie pour les résultats
    output_dir = "resultats_analyse_bandes"
    os.makedirs(output_dir, exist_ok=True)
    os.chdir(output_dir)
    
    # 1. Charger les données
    donnees_hyperspectrales, verite_terrain = charger_donnees(dataset_path)
    
    # 2. Préparer les données
    pixels, classes, classes_uniques, class_names = preparer_donnees(donnees_hyperspectrales, verite_terrain)
    
    # 3. Calculer la séparabilité par paires
    print("\nCalcul de la séparabilité entre paires de classes pour chaque bande...")
    resultats_paires = calculer_separabilite_paires_classes(pixels, classes, classes_uniques, class_names)
    
    # 4. Afficher quelques statistiques
    n_paires = len(set(resultats_paires['ClasseA'].astype(str) + "_" + resultats_paires['ClasseB'].astype(str)))
    n_bandes = len(set(resultats_paires['Bande']))
    
    print(f"Analyse terminée. Calculé la séparabilité pour {n_paires} paires de classes à travers {n_bandes} bandes.")
    print(f"Dimensions du tableau de résultats: {resultats_paires.shape}")
    
    # 5. Exporter les résultats complets au format CSV
    resultats_paires.to_csv('separabilite_paires_classes.csv', index=False)
    print("Résultats exportés dans 'separabilite_paires_classes.csv'")
    
    # 6. Analyser les bandes par leur worst-case et afficher les TOP 20
    print("\nAnalyse des bandes par leur worst-case...")
    worst_case_par_bande = analyser_worst_case(resultats_paires, top_n=20)
    
    # 7. Sélectionner les meilleures bandes selon le critère worst-case
    n_bandes_a_selectionner = 20  # Nombre de bandes à sélectionner
    bandes_selectionnees = worst_case_par_bande.head(n_bandes_a_selectionner)['Bande'].tolist()
    
    # 8. Sauvegarder la liste des bandes sélectionnées
    np.savetxt('bandes_selectionnees.txt', bandes_selectionnees, fmt='%d')
    print(f"\nLes {n_bandes_a_selectionner} meilleures bandes ont été sélectionnées et sauvegardées dans 'bandes_selectionnees.txt'")
    
    print("\nAnalyse terminée avec succès! Vous pouvez télécharger les fichiers suivants:")
    print("- separabilite_paires_classes.csv : Tableau complet des paires de classes pour chaque bande")
    print("- worst_case_par_bande.csv : Tableau des worst-case par bande")
    print("- bandes_selectionnees.txt : Liste des meilleures bandes sélectionnées")
    
    return resultats_paires, worst_case_par_bande, bandes_selectionnees

# Exécuter le programme principal
if __name__ == "__main__":
    resultats_paires, worst_case_par_bande, bandes_selectionnees = main()

Chargement des données hyperspectrales...
Dimensions de l'image hyperspectrale: (145, 145, 200)
Dimensions de la vérité terrain: (145, 145)
Nombre de classes uniques dans la vérité terrain: 17
Classes uniques: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
Données préparées: 21025 pixels avec 200 bandes
Nombre de classes (avec background): 17
Classe 0 (Background): 10776 pixels
Classe 1 (Alfalfa): 46 pixels
Classe 2 (Corn-notill): 1428 pixels
Classe 3 (Corn-mintill): 830 pixels
Classe 4 (Corn): 237 pixels
Classe 5 (Grass-pasture): 483 pixels
Classe 6 (Grass-trees): 730 pixels
Classe 7 (Grass-pasture-mowed): 28 pixels
Classe 8 (Hay-windrowed): 478 pixels
Classe 9 (Oats): 20 pixels
Classe 10 (Soybean-notill): 972 pixels
Classe 11 (Soybean-mintill): 2455 pixels
Classe 12 (Soybean-clean): 593 pixels
Classe 13 (Wheat): 205 pixels
Classe 14 (Woods): 1265 pixels
Classe 15 (Buildings-Grass-Trees-Drives): 386 pixels
Classe 16 (Stone-Steel-Towers): 93 pixels

Calcul de la séparabilité entr

Calcul de séparabilité par paires:  14%|█▍        | 3781/27200 [00:00<00:04, 5759.01it/s]/tmp/ipykernel_31/1229269973.py:18: RuntimeWarning: overflow encountered in scalar subtract
  c = b - a
Calcul de séparabilité par paires: 100%|██████████| 27200/27200 [00:03<00:00, 7542.55it/s]


Analyse terminée. Calculé la séparabilité pour 136 paires de classes à travers 200 bandes.
Dimensions du tableau de résultats: (27200, 7)
Résultats exportés dans 'separabilite_paires_classes.csv'

Analyse des bandes par leur worst-case...
Identification du worst-case pour chaque bande...

Top 20 des bandes selon leur pire cas de séparabilité:
     Bande  Separabilite  Chevauchement
44      44      0.071371       0.928629
142    142      0.063063       0.936937
47      47      0.055490       0.944510
38      38      0.054247       0.945753
48      48      0.044321       0.955679
52      52      0.039662       0.960338
51      51      0.038882       0.961118
102    102      0.037975       0.962025
40      40      0.037552       0.962448
42      42      0.036434       0.963566
49      49      0.035108       0.964892
104    104      0.033898       0.966102
50      50      0.032051       0.967949
41      41      0.031207       0.968793
37      37      0.029887       0.970113
43      43     

### Étape 7: Classification multiclasse avec MLP en utilisant différents ensembles de bandes discriminantes

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, BatchNormalization, Input, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import os
from scipy.io import loadmat
from tqdm import tqdm

# Définir explicitement les noms des classes
class_names_custom = [
    'Background',
    'Alfalfa',
    'Corn-notill',
    'Corn-mintill',
    'Corn',
    'Grass-pasture',
    'Grass-trees',
    'Grass-pasture-mowed',
    'Hay-windrowed',
    'Oats',
    'Soybean-notill',
    'Soybean-mintill',
    'Soybean-clean',
    'Wheat',
    'Woods',
    'Buildings-Grass-Trees-Drives',
    'Stone-Steel-Towers'
]

# Définir le chemin du dataset
dataset_path = "/kaggle/input/dataset-indian"

# Charger les données hyperspectrales et vérités terrain
def charger_donnees_pour_mlp():
    print("Chargement des données pour le MLP...")
    # Charger l'image hyperspectrale
    chemin_image = os.path.join(dataset_path, "Indian_pines_corrected.mat")
    donnees_mat = loadmat(chemin_image)
    donnees_hyperspectrales = donnees_mat['indian_pines_corrected']
    
    # Charger la vérité terrain
    chemin_gt = os.path.join(dataset_path, "Indian_pines_gt.mat")
    gt_mat = loadmat(chemin_gt)
    verite_terrain = gt_mat['indian_pines_gt']
    
    # Réorganiser les données
    height, width, n_bands = donnees_hyperspectrales.shape
    pixels = donnees_hyperspectrales.reshape(height * width, n_bands)
    classes = verite_terrain.reshape(height * width)
    
    print(f"Dimensions des données: pixels {pixels.shape}, classes {classes.shape}")
    return pixels, classes

# Charger les données
pixels, classes = charger_donnees_pour_mlp()

# Charger les résultats worst-case
print("Chargement des résultats worst-case...")
worst_case_par_bande = pd.read_csv('worst_case_par_bande.csv')

# Sélection des meilleures bandes selon les différentes configurations
print("Sélection des meilleures bandes selon différentes configurations...")
top5_bandes = worst_case_par_bande.head(5)['Bande'].values
top10_bandes = worst_case_par_bande.head(10)['Bande'].values
top15_bandes = worst_case_par_bande.head(15)['Bande'].values
top20_bandes = worst_case_par_bande.head(20)['Bande'].values

print(f"Top 5 bandes: {top5_bandes}")
print(f"Top 10 bandes: {top10_bandes}")
print(f"Top 15 bandes: {top15_bandes}")
print(f"Top 20 bandes: {top20_bandes}")

# 1. Fonction pour préparer les données selon les bandes sélectionnées
def preparer_donnees_mlp(pixels, classes, bandes_selectionnees):
    """
    Prépare les données pour l'entraînement avec les bandes sélectionnées.
    Inclut la classe de fond (background).
    
    Args:
        pixels: Données spectrales (n_pixels x n_bandes)
        classes: Étiquettes de classe pour chaque pixel
        bandes_selectionnees: Liste des indices des bandes à utiliser
    
    Returns:
        X_train, X_test, y_train, y_test: Ensembles d'entraînement et de test
    """
    # Utiliser tous les pixels, y compris la classe de fond (0)
    X = pixels   # Toutes les bandes
    y = classes  # Étiquettes (garder les indices originaux y compris 0)
    
    # Nombre de classes (incluant le fond)
    n_classes = len(np.unique(y))
    print(f"Nombre de classes (avec background): {n_classes}")
    
    # Sélectionner uniquement les bandes choisies
    X_selected = X[:, bandes_selectionnees]
    print(f"Dimensions des données: {X_selected.shape}")
    
    # Standardisation des données
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    
    # Conversion des étiquettes en format one-hot
    y_onehot = to_categorical(y)
    
    # Diviser en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_onehot, test_size=0.3, random_state=42, stratify=y
    )
    
    print(f"Ensemble d'entraînement: {X_train.shape}, {y_train.shape}")
    print(f"Ensemble de test: {X_test.shape}, {y_test.shape}")
    
    return X_train, X_test, y_train, y_test, n_classes

# 2. Définition du modèle MLP multiclasse avec BatchNorm avant activation
def creer_modele_mlp_multiclasse(input_dim, n_classes):
    """
    Crée un modèle MLP pour la classification multiclasse avec architecture 512-64-64-32
    et BatchNorm avant activation.
    """
    inputs = Input(shape=(input_dim,))
    
    # Première couche cachée
    x = Dense(512)(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Deuxième couche cachée
    x = Dense(64)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)
    
    # Troisième couche cachée
    x = Dense(64)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Quatrième couche cachée
    x = Dense(32)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Couche de sortie avec softmax pour la classification multiclasse
    outputs = Dense(n_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    # Compiler le modèle
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# 3. Fonction pour entraîner et évaluer un modèle
def entrainer_evaluer_modele(X_train, X_test, y_train, y_test, n_classes, 
                           bandes_selectionnees, nom_modele):
    """
    Entraîne et évalue un modèle MLP avec les données fournies.
    
    Args:
        X_train, X_test, y_train, y_test: Ensembles d'entraînement et de test
        n_classes: Nombre de classes
        bandes_selectionnees: Liste des indices des bandes utilisées
        nom_modele: Nom pour sauvegarder le modèle et les résultats
    
    Returns:
        model: Le modèle entraîné
        history: L'historique d'entraînement
        metrics: Dictionnaire des métriques d'évaluation
    """
    # Créer un dossier pour les résultats de ce modèle
    os.makedirs(f"resultats_{nom_modele}", exist_ok=True)
    
    print(f"\nCréation et entraînement du modèle {nom_modele}...")
    start_time = time.time()
    
    # Créer le modèle
    model = creer_modele_mlp_multiclasse(input_dim=len(bandes_selectionnees), n_classes=n_classes)
    model.summary()
    
    # Définir l'early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    )
    
    # Entraîner le modèle
    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=100,
        batch_size=32,
        callbacks=[early_stopping],
        verbose=1
    )
    
    train_time = time.time() - start_time
    print(f"\nTemps d'entraînement: {train_time:.2f} secondes")
    
    # Évaluation du modèle
    print("\nÉvaluation du modèle sur l'ensemble de test...")
    start_time = time.time()
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    predict_time = time.time() - start_time
    
    print(f"Précision (accuracy): {accuracy:.4f}")
    print(f"Temps de prédiction: {predict_time:.2f} secondes")
    
    # Générer les prédictions
    y_pred_prob = model.predict(X_test, verbose=0)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_true = np.argmax(y_test, axis=1)
    
    # Utiliser tous les noms de classes, y compris le background
    class_labels = class_names_custom[:n_classes]
    
    # Rapport de classification détaillé
    report = classification_report(y_true, y_pred, target_names=class_labels, zero_division=0)
    print("\nRapport de classification:")
    print(report)
    
    # Sauvegarder le rapport dans un fichier
    with open(f"resultats_{nom_modele}/rapport_classification_{nom_modele}.txt", "w") as f:
        f.write(f"Précision (accuracy): {accuracy:.4f}\n")
        f.write(f"Temps d'entraînement: {train_time:.2f} secondes\n")
        f.write(f"Temps de prédiction: {predict_time:.2f} secondes\n\n")
        f.write(report)
    
    # Visualisations
    # Courbes d'apprentissage
    plt.figure(figsize=(12, 5))
    
    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(f"resultats_{nom_modele}/learning_curves_{nom_modele}.png")
    plt.close()
    
    # Matrice de confusion
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(16, 14))
    
    # Utiliser des étiquettes sécurisées pour les axes
    x_labels = [label[:10] for label in class_labels]
    y_labels = [label[:10] for label in class_labels]
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=x_labels, 
                yticklabels=y_labels)
    plt.title(f'Matrice de confusion - {nom_modele}')
    plt.xlabel('Prédit')
    plt.ylabel('Réel')
    plt.tight_layout()
    plt.savefig(f"resultats_{nom_modele}/confusion_matrix_{nom_modele}.png")
    plt.close()
    
    # Précision par classe
    # Récupérer le rapport sous forme de dictionnaire
    report_dict = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
    
    # Préparer les données pour la visualisation
    precision_by_class = []
    for i, classe in enumerate(class_labels):
        if classe in report_dict:
            classe_dict = report_dict[classe]
            precision_by_class.append({
                'Classe': classe,
                'Précision': classe_dict['precision'],
                'Rappel': classe_dict['recall'],
                'F1-score': classe_dict['f1-score'],
                'Support': classe_dict['support']
            })
    
    # Créer le DataFrame et trier
    precision_df = pd.DataFrame(precision_by_class)
    if not precision_df.empty:
        precision_df = precision_df.sort_values('F1-score', ascending=False)
        
        # Visualisation
        plt.figure(figsize=(14, 8))
        sns.barplot(x='Classe', y='F1-score', data=precision_df)
        plt.title(f'F1-score par classe - {nom_modele}')
        plt.xlabel('Classe')
        plt.ylabel('F1-score')
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.savefig(f"resultats_{nom_modele}/f1score_by_class_{nom_modele}.png")
        plt.close()
        
        # Sauvegarder les F1-scores par classe
        precision_df.to_csv(f"resultats_{nom_modele}/f1scores_{nom_modele}.csv", index=False)
    else:
        print("Impossible de créer la visualisation du F1-score par classe - données insuffisantes")
    
    # Sauvegarde du modèle
    model.save(f"resultats_{nom_modele}/model_{nom_modele}.h5")
    print(f"\nModèle sauvegardé sous 'resultats_{nom_modele}/model_{nom_modele}.h5'")
    
    # Enregistrer les informations sur les bandes sélectionnées
    pd.DataFrame({
        'Bande': bandes_selectionnees,
        'Separabilite': [float(worst_case_par_bande[worst_case_par_bande['Bande'] == b]['Separabilite'].values[0]) 
                         for b in bandes_selectionnees]
    }).to_csv(f"resultats_{nom_modele}/bandes_selectionnees_{nom_modele}.csv", index=False)
    
    # Rassembler les métriques pour la comparaison finale
    metrics = {
        'accuracy': accuracy,
        'train_time': train_time,
        'predict_time': predict_time,
        'n_bands': len(bandes_selectionnees)
    }
    
    return model, history, metrics

# 4. Fonction principale pour entraîner tous les modèles
def entrainer_tous_modeles():
    """
    Fonction principale pour entraîner et évaluer tous les modèles.
    """
    # Créer un dossier de sortie pour les résultats globaux
    os.makedirs("resultats_comparaison", exist_ok=True)
    
    # Liste pour stocker les métriques de tous les modèles
    all_metrics = []
    
    # 1. Modèle avec Top 5 bandes
    print("\n========== MODÈLE TOP 5 BANDES ==========")
    X_train, X_test, y_train, y_test, n_classes = preparer_donnees_mlp(pixels, classes, top5_bandes)
    _, _, metrics_top5 = entrainer_evaluer_modele(
        X_train, X_test, y_train, y_test, n_classes, top5_bandes, "top5"
    )
    metrics_top5['model'] = 'Top 5 bandes'
    all_metrics.append(metrics_top5)
    
    # 2. Modèle avec Top 10 bandes
    print("\n========== MODÈLE TOP 10 BANDES ==========")
    X_train, X_test, y_train, y_test, n_classes = preparer_donnees_mlp(pixels, classes, top10_bandes)
    _, _, metrics_top10 = entrainer_evaluer_modele(
        X_train, X_test, y_train, y_test, n_classes, top10_bandes, "top10"
    )
    metrics_top10['model'] = 'Top 10 bandes'
    all_metrics.append(metrics_top10)
    
    # 3. Modèle avec Top 15 bandes
    print("\n========== MODÈLE TOP 15 BANDES ==========")
    X_train, X_test, y_train, y_test, n_classes = preparer_donnees_mlp(pixels, classes, top15_bandes)
    _, _, metrics_top15 = entrainer_evaluer_modele(
        X_train, X_test, y_train, y_test, n_classes, top15_bandes, "top15"
    )
    metrics_top15['model'] = 'Top 15 bandes'
    all_metrics.append(metrics_top15)
    
    # 4. Modèle avec Top 20 bandes
    print("\n========== MODÈLE TOP 20 BANDES ==========")
    X_train, X_test, y_train, y_test, n_classes = preparer_donnees_mlp(pixels, classes, top20_bandes)
    _, _, metrics_top20 = entrainer_evaluer_modele(
        X_train, X_test, y_train, y_test, n_classes, top20_bandes, "top20"
    )
    metrics_top20['model'] = 'Top 20 bandes'
    all_metrics.append(metrics_top20)
    
    # Créer un tableau de comparaison
    comparison_df = pd.DataFrame(all_metrics)
    comparison_df = comparison_df[['model', 'n_bands', 'accuracy', 'train_time', 'predict_time']]
    comparison_df.columns = ['Modèle', 'Nombre de bandes', 'Précision', 'Temps d\'entraînement (s)', 'Temps de prédiction (s)']
    
    # Sauvegarder le tableau de comparaison
    comparison_df.to_csv("resultats_comparaison/comparaison_modeles.csv", index=False)
    print("\nTableau de comparaison sauvegardé dans 'resultats_comparaison/comparaison_modeles.csv'")
    
    # Visualiser la comparaison des précisions
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Modèle', y='Précision', data=comparison_df)
    plt.title('Comparaison de la précision des modèles')
    plt.xlabel('Modèle')
    plt.ylabel('Précision')
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.savefig("resultats_comparaison/comparaison_precision.png")
    plt.close()
    
    # Visualiser la comparaison des temps d'entraînement
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Modèle', y='Temps d\'entraînement (s)', data=comparison_df)
    plt.title('Comparaison des temps d\'entraînement')
    plt.xlabel('Modèle')
    plt.ylabel('Temps d\'entraînement (s)')
    plt.tight_layout()
    plt.savefig("resultats_comparaison/comparaison_temps_entrainement.png")
    plt.close()
    
    print("\nAnalyse comparative terminée!")
    print("\nRécapitulatif des précisions:")
    for metric in all_metrics:
        print(f"{metric['model']}: {metric['accuracy']:.4f}")
    
    return comparison_df

# Exécuter l'entraînement des modèles
if __name__ == "__main__":
    comparison_results = entrainer_tous_modeles()

Chargement des données pour le MLP...
Dimensions des données: pixels (21025, 200), classes (21025,)
Chargement des résultats worst-case...
Sélection des meilleures bandes selon différentes configurations...
Top 5 bandes: [ 44 142  47  38  48]
Top 10 bandes: [ 44 142  47  38  48  52  51 102  40  42]
Top 15 bandes: [ 44 142  47  38  48  52  51 102  40  42  49 104  50  41  37]
Top 20 bandes: [ 44 142  47  38  48  52  51 102  40  42  49 104  50  41  37  43 144 145
  45  93]

========== MODÈLE TOP 5 BANDES ==========
Nombre de classes (avec background): 17
Dimensions des données: (21025, 5)
Ensemble d'entraînement: (14717, 5), (14717, 17)
Ensemble de test: (6308, 5), (6308, 17)

Création et entraînement du modèle top5...


I0000 00:00:1746635061.320227      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746635061.320897      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 5)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │           3,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │          32,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 17)                  │             561 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 45,393 (177.32 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 512)                 │          10,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_12 (Activation)           │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 64)                  │          32,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_13 (Activation)           │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_14 (Activation)           │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_15 (Activation)           │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 17)                  │             561 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 53,073 (207.32 KB)

 Trainable params: 51,729 (202.07 KB)

 Non-trainable params: 1,344 (5.25 KB)

Epoch 1/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.4015 - loss: 2.0437 - val_accuracy: 0.6067 - val_loss: 1.2172
Epoch 2/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5785 - loss: 1.2348 - val_accuracy: 0.6050 - val_loss: 1.1146
Epoch 3/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5917 - loss: 1.1571 - val_accuracy: 0.6033 - val_loss: 1.0853
Epoch 4/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5958 - loss: 1.1245 - val_accuracy: 0.6168 - val_loss: 1.0415
Epoch 5/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5973 - loss: 1.1059 - val_accuracy: 0.6315 - val_loss: 1.0441
Epoch 6/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5946 - loss: 1.0991 - val_accuracy: 0.6352 - val_loss: 1.0091
Epoch 7/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6033 - loss: 1.0702 - val_accuracy: 0.6165 - val_loss: 1.0266
Epoch 8/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5964 - loss: 1.0747 - val_accu

### Étape 8: Classification multiclasse avec MLP en utilisant des bandes discriminantes par segments égaux

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, BatchNormalization, Input, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import os
from scipy.io import loadmat
from tqdm import tqdm

# Définir explicitement les noms des classes
class_names_custom = [
    'Background',
    'Alfalfa',
    'Corn-notill',
    'Corn-mintill',
    'Corn',
    'Grass-pasture',
    'Grass-trees',
    'Grass-pasture-mowed',
    'Hay-windrowed',
    'Oats',
    'Soybean-notill',
    'Soybean-mintill',
    'Soybean-clean',
    'Wheat',
    'Woods',
    'Buildings-Grass-Trees-Drives',
    'Stone-Steel-Towers'
]

# Définir le chemin du dataset
dataset_path = "/kaggle/input/dataset-indian"

# Charger les données hyperspectrales et vérités terrain
def charger_donnees_pour_mlp():
    print("Chargement des données pour le MLP...")
    # Charger l'image hyperspectrale
    chemin_image = os.path.join(dataset_path, "Indian_pines_corrected.mat")
    donnees_mat = loadmat(chemin_image)
    donnees_hyperspectrales = donnees_mat['indian_pines_corrected']
    
    # Charger la vérité terrain
    chemin_gt = os.path.join(dataset_path, "Indian_pines_gt.mat")
    gt_mat = loadmat(chemin_gt)
    verite_terrain = gt_mat['indian_pines_gt']
    
    # Réorganiser les données
    height, width, n_bands = donnees_hyperspectrales.shape
    pixels = donnees_hyperspectrales.reshape(height * width, n_bands)
    classes = verite_terrain.reshape(height * width)
    
    print(f"Dimensions des données: pixels {pixels.shape}, classes {classes.shape}")
    return pixels, classes

# Charger les données
pixels, classes = charger_donnees_pour_mlp()

# Charger les résultats worst-case
print("Chargement des résultats worst-case...")
worst_case_par_bande = pd.read_csv('worst_case_par_bande.csv')

# Fonction pour sélectionner les meilleures bandes par segments égaux
def selectionner_meilleures_bandes_par_segment(worst_case_df, nb_segments):
    """
    Sélectionne la meilleure bande (selon le critère worst-case) dans chaque segment spectral.
    
    Args:
        worst_case_df: DataFrame contenant les résultats worst-case pour chaque bande
        nb_segments: Nombre de segments spectraux à considérer
    
    Returns:
        Liste des indices des bandes sélectionnées
    """
    # Nombre total de bandes
    nb_bandes_total = len(worst_case_df)
    
    # Taille approximative de chaque segment
    taille_segment = nb_bandes_total // nb_segments
    
    bandes_selectionnees = []
    
    # Pour chaque segment spectral
    for i in range(nb_segments):
        # Calculer les limites du segment
        debut = i * taille_segment
        fin = min((i + 1) * taille_segment - 1, nb_bandes_total - 1)
        
        # Sélectionner les bandes dans ce segment
        segment_df = worst_case_df[(worst_case_df['Bande'] >= debut) & (worst_case_df['Bande'] <= fin)]
        
        # Trier le segment par séparabilité décroissante
        segment_df = segment_df.sort_values('Separabilite', ascending=False)
        
        # Trouver la bande avec la meilleure séparabilité dans ce segment
        if not segment_df.empty:
            meilleure_bande = segment_df.iloc[0]['Bande']
            bandes_selectionnees.append(int(meilleure_bande))
    
    return bandes_selectionnees

# Sélection des meilleures bandes selon les différentes configurations de segments
print("Sélection des meilleures bandes selon différentes configurations de segments...")
equal5_bandes = selectionner_meilleures_bandes_par_segment(worst_case_par_bande, 5)
equal10_bandes = selectionner_meilleures_bandes_par_segment(worst_case_par_bande, 10)
equal15_bandes = selectionner_meilleures_bandes_par_segment(worst_case_par_bande, 15)
equal20_bandes = selectionner_meilleures_bandes_par_segment(worst_case_par_bande, 20)

print(f"Equal 5 bandes (1 par segment): {equal5_bandes}")
print(f"Equal 10 bandes (1 par segment): {equal10_bandes}")
print(f"Equal 15 bandes (1 par segment): {equal15_bandes}")
print(f"Equal 20 bandes (1 par segment): {equal20_bandes}")

# 1. Fonction pour préparer les données selon les bandes sélectionnées
def preparer_donnees_mlp(pixels, classes, bandes_selectionnees):
    """
    Prépare les données pour l'entraînement avec les bandes sélectionnées.
    Inclut la classe de fond (background).
    
    Args:
        pixels: Données spectrales (n_pixels x n_bandes)
        classes: Étiquettes de classe pour chaque pixel
        bandes_selectionnees: Liste des indices des bandes à utiliser
    
    Returns:
        X_train, X_test, y_train, y_test: Ensembles d'entraînement et de test
    """
    # Utiliser tous les pixels, y compris la classe de fond (0)
    X = pixels   # Toutes les bandes
    y = classes  # Étiquettes (garder les indices originaux y compris 0)
    
    # Nombre de classes (incluant le fond)
    n_classes = len(np.unique(y))
    print(f"Nombre de classes (avec background): {n_classes}")
    
    # Sélectionner uniquement les bandes choisies
    X_selected = X[:, bandes_selectionnees]
    print(f"Dimensions des données: {X_selected.shape}")
    
    # Standardisation des données
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    
    # Conversion des étiquettes en format one-hot
    y_onehot = to_categorical(y)
    
    # Diviser en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_onehot, test_size=0.3, random_state=42, stratify=y
    )
    
    print(f"Ensemble d'entraînement: {X_train.shape}, {y_train.shape}")
    print(f"Ensemble de test: {X_test.shape}, {y_test.shape}")
    
    return X_train, X_test, y_train, y_test, n_classes

# 2. Définition du modèle MLP multiclasse avec BatchNorm avant activation
def creer_modele_mlp_multiclasse(input_dim, n_classes):
    """
    Crée un modèle MLP pour la classification multiclasse avec architecture 512-64-64-32
    et BatchNorm avant activation.
    """
    inputs = Input(shape=(input_dim,))
    
    # Première couche cachée
    x = Dense(512)(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Deuxième couche cachée
    x = Dense(64)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)
    
    # Troisième couche cachée
    x = Dense(64)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Quatrième couche cachée
    x = Dense(32)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Couche de sortie avec softmax pour la classification multiclasse
    outputs = Dense(n_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    # Compiler le modèle
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# 3. Fonction pour entraîner et évaluer un modèle
def entrainer_evaluer_modele(X_train, X_test, y_train, y_test, n_classes, 
                           bandes_selectionnees, nom_modele):
    """
    Entraîne et évalue un modèle MLP avec les données fournies.
    
    Args:
        X_train, X_test, y_train, y_test: Ensembles d'entraînement et de test
        n_classes: Nombre de classes
        bandes_selectionnees: Liste des indices des bandes utilisées
        nom_modele: Nom pour sauvegarder le modèle et les résultats
    
    Returns:
        model: Le modèle entraîné
        history: L'historique d'entraînement
        metrics: Dictionnaire des métriques d'évaluation
    """
    # Créer un dossier pour les résultats de ce modèle
    os.makedirs(f"resultats_{nom_modele}", exist_ok=True)
    
    print(f"\nCréation et entraînement du modèle {nom_modele}...")
    start_time = time.time()
    
    # Créer le modèle
    model = creer_modele_mlp_multiclasse(input_dim=len(bandes_selectionnees), n_classes=n_classes)
    model.summary()
    
    # Définir l'early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    )
    
    # Entraîner le modèle
    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=100,
        batch_size=32,
        callbacks=[early_stopping],
        verbose=1
    )
    
    train_time = time.time() - start_time
    print(f"\nTemps d'entraînement: {train_time:.2f} secondes")
    
    # Évaluation du modèle
    print("\nÉvaluation du modèle sur l'ensemble de test...")
    start_time = time.time()
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    predict_time = time.time() - start_time
    
    print(f"Précision (accuracy): {accuracy:.4f}")
    print(f"Temps de prédiction: {predict_time:.2f} secondes")
    
    # Générer les prédictions
    y_pred_prob = model.predict(X_test, verbose=0)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_true = np.argmax(y_test, axis=1)
    
    # Utiliser tous les noms de classes, y compris le background
    class_labels = class_names_custom[:n_classes]
    
    # Rapport de classification détaillé
    report = classification_report(y_true, y_pred, target_names=class_labels, zero_division=0)
    print("\nRapport de classification:")
    print(report)
    
    # Sauvegarder le rapport dans un fichier
    with open(f"resultats_{nom_modele}/rapport_classification_{nom_modele}.txt", "w") as f:
        f.write(f"Précision (accuracy): {accuracy:.4f}\n")
        f.write(f"Temps d'entraînement: {train_time:.2f} secondes\n")
        f.write(f"Temps de prédiction: {predict_time:.2f} secondes\n\n")
        f.write(report)
    
    # Visualisations
    # Courbes d'apprentissage
    plt.figure(figsize=(12, 5))
    
    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(f"resultats_{nom_modele}/learning_curves_{nom_modele}.png")
    plt.close()
    
    # Matrice de confusion
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(16, 14))
    
    # Utiliser des étiquettes sécurisées pour les axes
    x_labels = [label[:10] for label in class_labels]
    y_labels = [label[:10] for label in class_labels]
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=x_labels, 
                yticklabels=y_labels)
    plt.title(f'Matrice de confusion - {nom_modele}')
    plt.xlabel('Prédit')
    plt.ylabel('Réel')
    plt.tight_layout()
    plt.savefig(f"resultats_{nom_modele}/confusion_matrix_{nom_modele}.png")
    plt.close()
    
    # Précision par classe
    # Récupérer le rapport sous forme de dictionnaire
    report_dict = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
    
    # Préparer les données pour la visualisation
    precision_by_class = []
    for i, classe in enumerate(class_labels):
        if classe in report_dict:
            classe_dict = report_dict[classe]
            precision_by_class.append({
                'Classe': classe,
                'Précision': classe_dict['precision'],
                'Rappel': classe_dict['recall'],
                'F1-score': classe_dict['f1-score'],
                'Support': classe_dict['support']
            })
    
    # Créer le DataFrame et trier
    precision_df = pd.DataFrame(precision_by_class)
    if not precision_df.empty:
        precision_df = precision_df.sort_values('F1-score', ascending=False)
        
        # Visualisation
        plt.figure(figsize=(14, 8))
        sns.barplot(x='Classe', y='F1-score', data=precision_df)
        plt.title(f'F1-score par classe - {nom_modele}')
        plt.xlabel('Classe')
        plt.ylabel('F1-score')
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.savefig(f"resultats_{nom_modele}/f1score_by_class_{nom_modele}.png")
        plt.close()
        
        # Sauvegarder les F1-scores par classe
        precision_df.to_csv(f"resultats_{nom_modele}/f1scores_{nom_modele}.csv", index=False)
    else:
        print("Impossible de créer la visualisation du F1-score par classe - données insuffisantes")
    
    # Sauvegarde du modèle
    model.save(f"resultats_{nom_modele}/model_{nom_modele}.h5")
    print(f"\nModèle sauvegardé sous 'resultats_{nom_modele}/model_{nom_modele}.h5'")
    
    # Enregistrer les informations sur les bandes sélectionnées
    pd.DataFrame({
        'Bande': bandes_selectionnees,
        'Segment': range(1, len(bandes_selectionnees) + 1),
        'Separabilite': [float(worst_case_par_bande[worst_case_par_bande['Bande'] == b]['Separabilite'].values[0]) 
                         for b in bandes_selectionnees]
    }).to_csv(f"resultats_{nom_modele}/bandes_selectionnees_{nom_modele}.csv", index=False)
    
    # Rassembler les métriques pour la comparaison finale
    metrics = {
        'accuracy': accuracy,
        'train_time': train_time,
        'predict_time': predict_time,
        'n_bands': len(bandes_selectionnees)
    }
    
    return model, history, metrics

# 4. Fonction principale pour entraîner tous les modèles
def entrainer_tous_modeles_equal_spacing():
    """
    Fonction principale pour entraîner et évaluer tous les modèles avec des bandes
    sélectionnées par segments égaux.
    """
    # Créer un dossier de sortie pour les résultats globaux
    os.makedirs("resultats_comparaison_equal", exist_ok=True)
    
    # Liste pour stocker les métriques de tous les modèles
    all_metrics = []
    
    # 1. Modèle avec 5 segments égaux (1 bande par segment)
    print("\n========== MODÈLE EQUAL 5 BANDES ==========")
    X_train, X_test, y_train, y_test, n_classes = preparer_donnees_mlp(pixels, classes, equal5_bandes)
    _, _, metrics_equal5 = entrainer_evaluer_modele(
        X_train, X_test, y_train, y_test, n_classes, equal5_bandes, "equal5"
    )
    metrics_equal5['model'] = '5 segments (5 bandes)'
    all_metrics.append(metrics_equal5)
    
    # 2. Modèle avec 10 segments égaux (1 bande par segment)
    print("\n========== MODÈLE EQUAL 10 BANDES ==========")
    X_train, X_test, y_train, y_test, n_classes = preparer_donnees_mlp(pixels, classes, equal10_bandes)
    _, _, metrics_equal10 = entrainer_evaluer_modele(
        X_train, X_test, y_train, y_test, n_classes, equal10_bandes, "equal10"
    )
    metrics_equal10['model'] = '10 segments (10 bandes)'
    all_metrics.append(metrics_equal10)
    
    # 3. Modèle avec 15 segments égaux (1 bande par segment)
    print("\n========== MODÈLE EQUAL 15 BANDES ==========")
    X_train, X_test, y_train, y_test, n_classes = preparer_donnees_mlp(pixels, classes, equal15_bandes)
    _, _, metrics_equal15 = entrainer_evaluer_modele(
        X_train, X_test, y_train, y_test, n_classes, equal15_bandes, "equal15"
    )
    metrics_equal15['model'] = '15 segments (15 bandes)'
    all_metrics.append(metrics_equal15)
    
    # 4. Modèle avec 20 segments égaux (1 bande par segment)
    print("\n========== MODÈLE EQUAL 20 BANDES ==========")
    X_train, X_test, y_train, y_test, n_classes = preparer_donnees_mlp(pixels, classes, equal20_bandes)
    _, _, metrics_equal20 = entrainer_evaluer_modele(
        X_train, X_test, y_train, y_test, n_classes, equal20_bandes, "equal20"
    )
    metrics_equal20['model'] = '20 segments (20 bandes)'
    all_metrics.append(metrics_equal20)
    
    # Créer un tableau de comparaison
    comparison_df = pd.DataFrame(all_metrics)
    comparison_df = comparison_df[['model', 'n_bands', 'accuracy', 'train_time', 'predict_time']]
    comparison_df.columns = ['Modèle', 'Nombre de bandes', 'Précision', 'Temps d\'entraînement (s)', 'Temps de prédiction (s)']
    
    # Sauvegarder le tableau de comparaison
    comparison_df.to_csv("resultats_comparaison_equal/comparaison_modeles_equal.csv", index=False)
    print("\nTableau de comparaison sauvegardé dans 'resultats_comparaison_equal/comparaison_modeles_equal.csv'")
    
    # Visualiser la comparaison des précisions
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Modèle', y='Précision', data=comparison_df)
    plt.title('Comparaison de la précision des modèles (Equal Spacing)')
    plt.xlabel('Modèle')
    plt.ylabel('Précision')
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.savefig("resultats_comparaison_equal/comparaison_precision_equal.png")
    plt.close()
    
    # Visualiser la comparaison des temps d'entraînement
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Modèle', y='Temps d\'entraînement (s)', data=comparison_df)
    plt.title('Comparaison des temps d\'entraînement (Equal Spacing)')
    plt.xlabel('Modèle')
    plt.ylabel('Temps d\'entraînement (s)')
    plt.tight_layout()
    plt.savefig("resultats_comparaison_equal/comparaison_temps_entrainement_equal.png")
    plt.close()
    
    # Visualiser la distribution des bandes sélectionnées
    plt.figure(figsize=(15, 8))
    
    # Créer une matrice pour représenter toutes les bandes
    all_bands = np.zeros(200)
    markers = ['o', 's', 'D', '^']
    colors = ['blue', 'green', 'red', 'purple']
    labels = ['5 segments', '10 segments', '15 segments', '20 segments']
    
    # Tracer la distribution des bandes pour chaque configuration
    for i, bandes in enumerate([equal5_bandes, equal10_bandes, equal15_bandes, equal20_bandes]):
        plt.scatter(bandes, np.ones(len(bandes))*i+1, marker=markers[i], 
                   color=colors[i], s=100, label=labels[i])
    
    # Ajouter des lignes verticales pour montrer les segments
    for i in range(1, 20):
        plt.axvline(x=i*10, color='gray', linestyle='--', alpha=0.3)
    
    plt.title('Distribution des bandes sélectionnées par l\'approche Equal Spacing')
    plt.xlabel('Indice de bande')
    plt.yticks([1, 2, 3, 4], labels)
    plt.xlim(-5, 205)
    plt.grid(axis='x', alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.savefig("resultats_comparaison_equal/distribution_bandes_equal.png")
    plt.close()
    
    print("\nAnalyse comparative terminée!")
    print("\nRécapitulatif des précisions:")
    for metric in all_metrics:
        print(f"{metric['model']}: {metric['accuracy']:.4f}")
    
    return comparison_df

# Exécuter l'entraînement des modèles avec sélection par segments égaux
if __name__ == "__main__":
    comparison_results = entrainer_tous_modeles_equal_spacing()

Chargement des données pour le MLP...
Dimensions des données: pixels (21025, 200), classes (21025,)
Chargement des résultats worst-case...
Sélection des meilleures bandes selon différentes configurations de segments...
Equal 5 bandes (1 par segment): [38, 44, 102, 142, 199]
Equal 10 bandes (1 par segment): [0, 38, 44, 60, 93, 102, 125, 142, 163, 199]
Equal 15 bandes (1 par segment): [0, 25, 38, 44, 52, 74, 84, 102, 104, 117, 142, 144, 159, 172, 194]
Equal 20 bandes (1 par segment): [0, 19, 27, 38, 44, 52, 60, 74, 84, 93, 102, 116, 125, 130, 142, 159, 163, 172, 180, 199]

========== MODÈLE EQUAL 5 BANDES ==========
Nombre de classes (avec background): 17
Dimensions des données: (21025, 5)
Ensemble d'entraînement: (14717, 5), (14717, 17)
Ensemble de test: (6308, 5), (6308, 17)

Création et entraînement du modèle equal5...


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 5)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 512)                 │           3,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_16 (Activation)           │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 64)                  │          32,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_17 (Activation)           │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_18 (Activation)           │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_19 (Activation)           │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 17)                  │             561 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 45,393 (177.32 KB)

 Trainable params: 44,049 (172.07 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 512)                 │          10,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_28               │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_28 (Activation)           │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_36 (Dense)                     │ (None, 64)                  │          32,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_29               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_29 (Activation)           │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_30               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_30 (Activation)           │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_38 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_31               │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_31 (Activation)           │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_39 (Dense)                     │ (None, 17)                  │             561 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 53,073 (207.32 KB)

 Trainable params: 51,729 (202.07 KB)

 Non-trainable params: 1,344 (5.25 KB)

Epoch 1/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.4776 - loss: 1.7990 - val_accuracy: 0.6145 - val_loss: 1.1321
Epoch 2/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5957 - loss: 1.1574 - val_accuracy: 0.6345 - val_loss: 1.0092
Epoch 3/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6180 - loss: 1.0797 - val_accuracy: 0.6624 - val_loss: 0.9539
Epoch 4/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6314 - loss: 1.0119 - val_accuracy: 0.6671 - val_loss: 0.8862
Epoch 5/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6494 - loss: 0.9557 - val_accuracy: 0.6749 - val_loss: 0.8774
Epoch 6/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6567 - loss: 0.9465 - val_accuracy: 0.6821 - val_loss: 0.8280
Epoch 7/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6646 - loss: 0.9026 - val_accuracy: 0.6712 - val_loss: 0.8479
Epoch 8/100
368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6751 - loss: 0.8727 - val_accu